In [4]:
from PIL import Image
import numpy
import random
import math

apple = Image.open("banner.png")
array = numpy.array(apple)


for i in range(len(array)):
    for j in range(len(array[i])):
        if array[i][j][0] > 150:
            array[i][j] = [0,min(255,2 * array[i][j][1]),min(255,2 * array[i][j][2]), 255]
        array[i][j][0] += random.uniform(-0.1, 0.1)
        array[i][j][1] += random.uniform(-0.1, 0.1)
        array[i][j][2] += random.uniform(-0.1, 0.1)
        array[i][j][3] += random.uniform(-0.1, 0.1)

apple2 = Image.fromarray(array)
apple2.show()

FileNotFoundError: [Errno 2] No such file or directory: 'banner.png'

In [1]:
from PIL import Image
import numpy
from Controller import converter
from Controller import slam
import cv2
import math

from matplotlib import pyplot

leftImg, rightImg = numpy.split(numpy.array(Image.open("./img.png")), 2, axis=1)
# leftImg = numpy.array(Image.open("./undistorted_img.png"))
def show(img):
    Image.fromarray(img).show()

# show(leftImg)

# filteredLeftImg = converter.filter(leftImg)
# leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)
new_K = converter.K.copy()
new_K[0][0] *= 0.5
new_K[1][1] *= 0.5
map1, map2 = cv2.fisheye.initUndistortRectifyMap(converter.K, converter.D, numpy.eye(3), new_K, (converter.imageWidth, converter.imageHeight), cv2.CV_16SC2)
undistortedLeftImg = cv2.remap(leftImg, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
leftImg = undistortedLeftImg
leftEdgesIn, gLeftIn, rLeftIn = converter.filter(cv2.cvtColor(undistortedLeftImg, cv2.COLOR_RGB2BGR))
# show(leftBlurredIn)
# show(leftEdgesIn)
# show(cv2.cvtColor(leftImg, cv2.COLOR_BGR2HSV))

# rightImg= leftImg
# filteredRightImg = converter.filter(rightImg)
# rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)
rightEdgesIn, gRightIn, rRightIn = converter.filter(cv2.cvtColor(rightImg, cv2.COLOR_RGB2BGR))
# show(rightEdgesIn)

leftHeights, rightHeights = converter.getHeights(leftEdgesIn, rightEdgesIn)

slam.findStartingPosition(leftHeights, rightHeights)

print(slam.carX, slam.carY)

rLeftBlobs, gLeftBlobs, rRightBlobs, gRightBlobs = converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn)
leftLandmarks = converter.getWallLandmarks(leftHeights, rLeftBlobs, gLeftBlobs)
rightLandmarks = converter.getWallLandmarks(rightHeights, rRightBlobs, gRightBlobs)

walls = []
# for landmark in leftLandmarks:
#     walls.append(converter.getDistance(landmark[0], leftHeights[landmark[0]], -1))
# print(walls)
# for landmark in rightLandmarks:
#     walls.append(converter.getDistance(landmark[0], rightHeights[landmark[0]], 1))

redBlobs = []
# for blob in rLeftBlobs:
#     redBlobs.append(converter.getDistance(blob[0], leftHeights[blob[0]], -1))
# for blob in rRightBlobs:
#     redBlobs.append(converter.getDistance(blob[0], rightHeights[blob[0]], 1))
greenBlobs = []
# for blob in rLeftBlobs:
#     greenBlobs.append(converter.getDistance(blob[0], leftHeights[blob[0]], -1))
# for blob in rRightBlobs:
#     greenBlobs.append(converter.getDistance(blob[0], rightHeights[blob[0]], 1))
# print(walls)
# slam.slam(walls, redBlobs, greenBlobs)

print(slam.carX, slam.carY, slam.carAngle)

# print(converter.getDistance(round(converter.imageWidth / 2), leftHeights[round(converter.imageWidth / 2)], -1))
for l in leftLandmarks:
    for i in range(-3, 3):
        for j in range(-3, 3):
            leftImg[converter.wallStartLeft][l[0]] = [255, 0, 0]
for i in range(converter.imageWidth):
    leftImg[int(converter.wallStartLeft + leftHeights[i])][i] = [0, 255, 0]
show(leftImg)
show(leftEdgesIn[converter.wallStartLeft:converter.wallEnd])
# print(gLeftBlobs)
# distances = converter.getDistances(leftEdgesIn, rightEdgesIn)
# x = []
# y = []
# for i in distances:
#     # print(i)
#     if i[2] < 200:
#         x.append(i[0])
#         y.append(i[1])
# # # pyplot.xlim(-50, 50)
# # # pyplot.ylim(-50, 50)
# x.append(0)
# y.append(0)
# pyplot.axes().set_aspect('equal')
# pyplot.scatter(x, y)


38.0 7.0 82.0
20 125
477 0.06666666666666667 -0.03333333333333333 3.155555555555553 8.572222222222223
43 0.2 0.1 10.199999999999962 21.949999999999992
44 0.2 0.1 4.799999999999987 26.150000000000006
45 0.2 0.1 2.800000000000045 30.95
46 0.2 0.1 3.200000000000022 36.34999999999999
47 0.2 0.06666666666666667 5.999999999999976 18.222222222222232
48 0.23333333333333334 0.06666666666666667 4.905555555555549 20.48888888888888
49 0.2 0.06666666666666667 4.000000000000005 23.022222222222215
50 0.2 0.06666666666666667 2.800000000000045 25.822222222222226
51 0.2 0.06666666666666667 4.000000000000005 28.888888888888893
52 0.2 0.06666666666666667 7.59999999999997 32.22222222222223
53 0.16666666666666666 0.03333333333333333 5.30555555555556 11.90555555555553
54 0.2 0.03333333333333333 4.000000000000004 12.772222222222204
55 0.2 0.03333333333333333 2.800000000000045 13.70555555555554
56 0.2 0.03333333333333333 4.000000000000005 14.705555555555568
57 0.2 0.03333333333333333 7.59999999999997 15.772222

In [2]:
from PIL import Image
import numpy
from Controller import converter
from Controller import slam
import cv2
import math

from matplotlib import pyplot

# leftImg, rightImg = numpy.split(numpy.array(Image.open("./undistorted_img.png")), 2, axis=1)
leftImg = numpy.array(Image.open("./undistorted_img.png"))
def show(img):
    Image.fromarray(img).show()

# show(leftImg)

# filteredLeftImg = converter.filter(leftImg)
# leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)
leftEdgesIn, gLeftIn, rLeftIn = converter.filter(cv2.cvtColor(leftImg, cv2.COLOR_RGB2BGR))
# show(leftBlurredIn)
# show(leftEdgesIn)
# show(cv2.cvtColor(leftImg, cv2.COLOR_BGR2HSV))

rightImg= leftImg
# filteredRightImg = converter.filter(rightImg)
# rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)
rightEdgesIn, gRightIn, rRightIn = converter.filter(cv2.cvtColor(rightImg, cv2.COLOR_RGB2BGR))
# show(rightEdgesIn)

distances = converter.getDistances(leftEdgesIn, rightEdgesIn)
x = []
y = []
for i in distances:
    # print(i)
    if i[2] < 200:
        x.append(i[0])
        y.append(i[1])
# # pyplot.xlim(-50, 50)
# # pyplot.ylim(-50, 50)
x.append(0)
y.append(0)
pyplot.axes().set_aspect('equal')
pyplot.scatter(x, y)

FileNotFoundError: [Errno 2] No such file or directory: './undistorted_img.png'

In [ ]:
from PIL import Image
import numpy
from Controller import converter
from Controller import slam
import cv2
import math

from matplotlib import pyplot

# leftImg, rightImg = numpy.split(numpy.array(Image.open("./undistorted_img.png")), 2, axis=1)
leftImg = numpy.array(Image.open("./undistorted_img.png"))
def show(img):
    Image.fromarray(img).show()

# show(leftImg)

# filteredLeftImg = converter.filter(leftImg)
# leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)
leftEdgesIn, gLeftIn, rLeftIn = converter.filter(cv2.cvtColor(leftImg, cv2.COLOR_RGB2BGR))
# show(leftBlurredIn)
# show(leftEdgesIn)
# show(cv2.cvtColor(leftImg, cv2.COLOR_BGR2HSV))

rightImg= leftImg
# filteredRightImg = converter.filter(rightImg)
# rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)
rightEdgesIn, gRightIn, rRightIn = converter.filter(cv2.cvtColor(rightImg, cv2.COLOR_RGB2BGR))
# show(rightEdgesIn)

# distances, croppedLeft, rawHeightsLeft = converter.getDistances(leftBlurredIn, leftEdgesIn, rightBlurredIn, rightEdgesIn)
leftHeights, rightHeights = converter.getHeights(leftEdgesIn, rightEdgesIn)
# x = round(converter.focalLength / math.sqrt(3) + converter.imageWidth / 2)
# print(leftHeights[x], x)
# x = round(converter.imageWidth / 2 - converter.focalLength * math.sqrt(3))
# print(leftHeights[x], x)
# show(croppedLeft)
# for i in rawHeightsLeft:
#     croppedLeft[i][rawHeightsLeft[i]] = 125
# show(croppedLeft)
# image = []

# slam.findStartingPosition(leftHeights, rightHeights)

# print(slam.carX, slam.carY)

# print(rawHeightsLeft)

# # # print(heights)
# # # print(left)
# # print(distances)
# x = []
# y = []
# for i in distances:
#     # print(i)
#     # if i[2] < 200:
#     x.append(i[0])
#     y.append(i[1])
# # pyplot.xlim(-50, 50)
# # pyplot.ylim(-50, 50)
# x.append(0)
# y.append(0)
# pyplot.axes().set_aspect('equal')
# pyplot.scatter(x, y)
# for i in range(len(rawHeightsLeft)):
#     print(rawHeightsLeft[i])
#     print(distances[i])

# for i in distances:
#     image.append([i[2], i[2], i[2]])
# show(numpy.array(image))
# show(numpy.concatenate(([distances], [distances], [distances]), axis=0))
# show(converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn))
rLeftBlobs, gLeftBlobs, rRightBlobs, gRightBlobs = converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn)
gLeftBlobs = []
leftLandmarks = converter.getWallLandmarks(leftHeights, rLeftBlobs, gLeftBlobs)
rightLandmarks = converter.getWallLandmarks(rightHeights, rRightBlobs, gRightBlobs)
# print(rLeftBlobs, gLeftBlobs)

# walls = []
# for landmark in leftLandmarks:
#     walls.append(converter.getDistance(landmark[0], leftHeights[landmark[0]], -1))
# for landmark in rightLandmarks:
#     walls.append(converter.getDistance(landmark[0], rightHeights[landmark[0]], 1))

# redBlobs = []
# for blob in rLeftBlobs:
#     redBlobs.append(converter.getDistance(blob[0], leftHeights[blob[0]], -1))
# for blob in rRightBlobs:
#     redBlobs.append(converter.getDistance(blob[0], rightHeights[blob[0]], 1))
# greenBlobs = []
# for blob in rLeftBlobs:
#     greenBlobs.append(converter.getDistance(blob[0], leftHeights[blob[0]], -1))
# for blob in rRightBlobs:
#     greenBlobs.append(converter.getDistance(blob[0], rightHeights[blob[0]], 1))
# print(walls)
# slam.slam(walls, redBlobs, greenBlobs)

# print(slam.carX, slam.carY, slam.carAngle)

# for blob in leftBlobs:
#     cv2.circle(leftImg, (blob[0], converter.wallStartLeft), blob[1], (255, 0, 0), 1)
# for blob in rightBlobs:
#     cv2.circle(rightImg, (blob[0], converter.wallStartRight), blob[1], (255, 0, 0), 1)

# corners = cv2.goodFeaturesToTrack(leftEdgesIn, 27, 0.5, 10)
# corners = numpy.int0(corners)
# print(corners)
# print(landmarks)
# for l in corners:
#     for i in range(-3, 3):
#         for j in range(-3, 3):
#             leftImg[l[0][1] + i][l[0][0] + j] = [255, 0, 0]
print(leftLandmarks)
for l in leftLandmarks:
    # leftEdgesIn[max(round(l[1]), 0)][l[0]] = 255
    # leftEdgesIn[round(converter.imageHeight / 2 + 15 + leftHeights[l[0]]) + 2][l[0]] = 255
    for i in range(-3, 3):
        for j in range(-3, 3):
            leftImg[converter.wallStartLeft][l[0] + i] = [255, 0, 0]
#     print(l[0], converter.getDistance(l[0], leftHeights[l[0]], -1))
# for l in landmarks:
#     # leftEdgesIn[max(round(l[1]), 0)][l[0]] = 255
#     # leftEdgesIn[round(converter.imageHeight / 2 + 15 + leftHeights[l[0]]) + 2][l[0]] = 255
#     for i in range(-3, 3):
#         for j in range(-3, 3):
#             rightImg[converter.wallStartRight][l[0] + i] = [255, 0, 0]
#     print(l[0], converter.getDistance(l[0], rightHeights[l[0]], -1))

# # show(leftEdgesIn)
show(leftImg)
# outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks = converter.getLandmarks(distances, rBlobs, gBlobs)

# # for landmark in outerWallLandmarks:
# #     if landmark[2] < 30:
# #         print(landmark)
# print(outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks)

# print(converter.verticalFocalLength, converter.horizontalFocalLength)

In [ ]:
import math

horizontalFov = 155
verticalFov = 115
imageWidth = 272
imageHeight = 154
focalLength = ((imageHeight / 2) / math.tan(math.pi * (verticalFov / 2) / 180))
wallHeight = 10
centerOffset = 10
dist = wallHeight * math.sqrt(focalLength**2 + (imageWidth / 2 - 29)**2) / 36
# return (dir * (3 + a[1] * dist), (10 + a[2] * dist), dist)

print(focalLength)
dist

49.05441008217698


32.69685768082119